# From TF to TF-lite

## Download the Tensorflow model

In [ ]:
# download the model and rename it
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [ ]:
# check python version
!python -V

In [ ]:
!pip install keras-image-helper

In [ ]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:
# Necessary import
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite_int # to avoid depending on tensorflow
from PIL import Image
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
from keras_image_helper import create_preprocessor

# Check tensorflow version
tf.__version__

In [ ]:
# Download a pants' image
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

In [ ]:
# Load the model
model = keras.models.load_model('clothing-model.h5')

In [ ]:
# Load an image
img = load_img('pants.jpg', target_size = (299, 299))

# Check the image
img

In [ ]:
# Convert the image to a numpy array
x = np.array(img)
# Batch of 1 image
X = np.array([x])

# Preprocess input batch
X = preprocess_input(X)

In [ ]:
# Shape of the batch
X.shape

In [ ]:
# Make prediction
preds = model.predict(X)

In [ ]:
# Visualize the predictions
preds

In [ ]:
# Target classes
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Classes and correponding probabilities
dict(zip(classes, preds[0]))

## Convert Keras to TF-Lite

In [ ]:
# Initialize converter for the tf model
converter = tflite.TFLiteConverter.from_keras_model(model)

# Convert to a tf-lite model
tflite_model = converter.convert()

# Save the converted model as a binary file
with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [ ]:
# list files to find the conerted model
!ls -lh

In [ ]:
# Load the model
interpreter = tflite.Interpreter(model_path = 'clothing-model.tflite')
# Load the weights from the model to memory (necessary with tf-lite models)
interpreter.allocate_tensors()

# Specify the input
input_index = interpreter.get_input_details()[0]['index']
# Specify the output
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
# Initialize the input of the interpreter
interpreter.set_tensor(input_index, X)
# Invoke computations in the neural network
interpreter.invoke()
# Fetch predictions
preds = interpreter.get_tensor(output_index)

Note that we are using the input batch `X`, that was preprocessed with the function `preprocess_input()`, and previously loaded using `load_img()`, both from `tensorflow`. We need to remove these tensorflow depenedencies.

In [ ]:
# Target classes
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Classes and predicted probabilities 
dict(zip(classes, preds[0]))

## Removing TF dependency

We use `Image` from the library `PIL` to load the image: 

In [ ]:
# read pants' image, resizing it
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

# Visualize the image
img

In [ ]:
# Function to preprocess the input batch of images
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [ ]:
# Convert the image to a numpy array
x = np.array(img, dtype = 'float32')
# Batch of 1 image
X = np.array([x])

# Preprocess input batch
X = preprocess_input(X)

In [ ]:
# Initialize the input of the interpreter
interpreter.set_tensor(input_index, X)
# Invoke computations in the neural network
interpreter.invoke()
# Fetch predictions
preds = interpreter.get_tensor(output_index)

In [ ]:
# Target classes
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Classes and predicted probabilities 
dict(zip(classes, preds[0]))

Now we do not have any dependency on tensorflow.

## Simpler way of doing it

We can use the function `create_preprocessor` from the library `keras_image_helper`. Note that instead of importing `tf-lite` from `tf`, we imported ` tflite_runtime.interpreter`.

In [ ]:
# Load the model
interpreter = tflite_int.Interpreter(model_path = 'clothing-model.tflite')
# Load the weights from the model to memory (necessary with tf-lite models)
interpreter.allocate_tensors()

# Specify the input
input_index = interpreter.get_input_details()[0]['index']
# Specify the output
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
# Initialize data preprocessor
preprocessor = create_preprocessor('xception', target_size = (299, 299))

In [ ]:
# image url
url = 'http://bit.ly/mlbookcamp-pants'
# preprocess image
X = preprocessor.from_url(url)

Note that we don't even have to download the image.

In [ ]:
# Initialize the input of the interpreter
interpreter.set_tensor(input_index, X)
# Invoke computations in the neural network
interpreter.invoke()
# Fetch predictions
preds = interpreter.get_tensor(output_index)

In [ ]:
# Target classes
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Classes and predicted probabilities 
dict(zip(classes, preds[0]))

---